In [2]:
# Carga paquetes
import pandas as pd
import numpy as np
import warnings
import seaborn as sn

warnings.filterwarnings("ignore", category=UserWarning)  # ignora los warnings

In [67]:
###########    FUNCIONES   ##########
## Interquartile Range (IQR)
#Se calculan los limites superiores e inferiores para identificación de posibles datos atipicos
# IQR = Q3 - Q1 
# LI = Q1–1.5 * IQR; LS = Q3+1.5 * IQR.
def IQR(variable):
  Iqr = np.percentile(variable,75)-np.percentile(variable,25)

  LI=np.percentile(variable,25)-(1.5*Iqr)
  LS=np.percentile(variable,75)+(1.5*Iqr)
  return(LI,LS)

In [69]:
###### RUTAS Y CARGA DE INFO  #####
inpath = "C:/Users/Diego Torres/OneDrive/Datasets/Manar/RFM_pruebas.xlsx"

tb = pd.read_excel(inpath, sheet_name = "Hoja2")

In [70]:
#################
# Descriptivos
print(tb.sample(5))
print(' ')
print('Summary:')
tb.info()
print(' ')
print('Descriptivos:')
print(tb.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(tb.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(tb.shape)


#outliers(tb[[]])

        codigo cliente  nombre cliente  codigo producto   
34853      13350006477     13350006477         32408002  \
107149     13310604273     13310604273         32802001   
37596     133110567073    133110567073         32408002   
95699      13330019816     13330019816         32104001   
10973        112036563       112036563         31705007   

                                          nombre producto fecha venta   
34853               DIF MT-IBC MENSUALIDAD CEN FINANCIERO  2022-03-23  \
107149  DIF MANTENIMIENTO A DISTANCIA DE PROGRAMAS Y E...  2022-08-25   
37596               DIF MT-IBC MENSUALIDAD CEN FINANCIERO  2022-03-23   
95699        MT-IBC MENSUALIDAD NUMERO TRANSACCIONES CENT  2022-08-02   
10973   DIF MANTENIMIENTO A DISTANCIA DE PROGRAMAS Y E...  2022-01-27   

        cantidad facturado  valor facturado               Tipo  
34853                    1          28614.0     CEN FINANCIERO  
107149                   1          47594.0  CEN TRANSACCIONAL  
37596      

In [123]:
rec = max(tb['fecha venta']) - min(tb['fecha venta'])
type(rec)

pandas._libs.tslibs.timedeltas.Timedelta

In [120]:
####   RFM CLIENTES
#.groupby(['Tipo','codigo cliente'])\
hoy = max(tb['fecha venta'])

tb_grp = tb\
    .groupby('codigo cliente')\
    .agg(
        dias_rec=('fecha venta', 'max'),
        Frecuencia=('cantidad facturado', 'sum'),
        Monto=('valor facturado', 'sum'),
        nombre_cliente=('nombre cliente', 'unique'))\
    .assign(recencia=lambda x: hoy - x['dias_rec'])\
    .sort_values('codigo cliente')
    #.assign(Recencia=(tb['recencia'].apply(lambda y: y.days)))

print(tb_grp.sample(5))
print(' ')
print('Dimensiones:')
print(tb_grp.shape)

                 dias_rec  Frecuencia       Monto nombre_cliente recencia
codigo cliente                                                           
112686376      2022-08-25          40   3426481.0    [112686376]   6 days
113271182      2022-08-25          66   6245319.0    [113271182]   6 days
112443407      2022-08-25          40  11654337.0    [112443407]   6 days
113210819      2022-08-02          16   7075403.0    [113210819]  29 days
113285578      2022-08-02          24   3991592.0    [113285578]  29 days
 
Dimensiones:
(3822, 5)


In [127]:
type(tb_grp['recencia']) #.apply(lambda y: y.days)

pandas.core.series.Series

In [75]:
### Pruebas
""" 
outliers = IQR(tb_grp['Monto'])
print(outliers)

variable = tb_grp['Monto']
#variable.name

if variable.name == 'Monto':
    LI = outliers[0]
    LS = outliers[1]
    VarTemp = variable[(variable >= LI) & (variable <= LS)]

print('Dimensiones:')
print(VarTemp.shape) """

(-6958763.0, 13796469.0)
Dimensiones:
(3294,)


In [74]:
""" # filter con lamtba asi:, el resultado debe ser una lista
len(list(filter(lambda x: x > 10724445.0,tb_grp['Monto']))) """

647

In [94]:
breaks_1 = np.concatenate(([min(variable) - 0.001]\
,np.percentile(VarTemp, np.arange(0, 101, 20)[1:5]) #np.arange(start, stop, step), stop excluded so +1\
,[max(variable) + 0.001]))

len(breaks_1)
breaks_1

array([-1.51483470e+07,  5.96995000e+05,  1.24082460e+06,  2.29349800e+06,
        4.58032560e+06,  5.43131119e+09])

In [104]:
def categorizacion (variable,outliers,reciencia):
  LI = outliers[0]
  LS = outliers[1]

  # variable tiene outliers, VarTemp no
  VarTemp = variable[(variable >= LI) & (variable <= LS)]

  ## TIPO DE INTERVALO O BREAK  
  # valores en el que se dividen los intervalos. 
  # los datos atipicos se apartan a la hora de definir los quantiles, pero se incluyen al final para el puntaje
  # break_1: <LI + min; percentiles -1 y 6; max + >LS
  breaks_1 = np.concatenate(([min(variable) - 0.001]\
    ,np.percentile(VarTemp, np.arange(0, 101, 20)[1:5]) #np.arange(start, stop, step), stop excluded so +1\
    ,[max(variable) + 0.001]))

  # break_2: quantiles con outliers
  quantiles_2 = np.percentile(variable, np.arange(0, 101, 20))
  quantiles_2[0] -= 0.001
  quantiles_2[-1] += 0.001
  breaks_2 = quantiles_2
 
  # break_3: Usandos rangos fijos y no quantiles incluyendo outliers
  breaks_3 = np.linspace(min(variable) - 0.001, max(variable) + 0.001, num=6)

  if not reciencia:
    # Usar el break sin outliers sin que quantiles repitan valores
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_1)
    # Usar el break con outliers sin que quantiles repitan valores
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_2)
    # Usar el break con outliers sin usar quantiles sino rangos fijos    
    else:
        puntaje = pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_3)

# restar 6 y multiplicar -1 invierte el puntaje para Recencia, es decir, si el puntaje mayor es 5 para los valores mas grandes 
# en los otros casos, en Recencia los menores son los de mayor puntaje
  else:
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_1)
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_2)
    else:
        puntaje = ((pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_3)

  return (puntaje,categorias)

In [129]:
print(' ')
print('Summary:')
tb_grp.info()

 
Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 3822 entries, 1141132 to 139110576375
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   dias_rec        3822 non-null   datetime64[ns] 
 1   Frecuencia      3822 non-null   int64          
 2   Monto           3822 non-null   float64        
 3   nombre_cliente  3822 non-null   object         
 4   recencia        3822 non-null   timedelta64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1), timedelta64[ns](1)
memory usage: 179.2+ KB


In [105]:
tb_RFM = puntajes_RFM(tb_grp)

#################
# Descriptivos
print(tb_RFM.sample(5))
print(' ')

TypeError: unsupported operand type(s) for -: 'Timestamp' and 'float'

In [102]:


def puntajes_RFM (tb):
  #Ejecutamos las dos funciones anteriores
  rfm_df = tb
  lim_rec = IQR(rfm_df['Recencia'])
  lim_freq = IQR(rfm_df['Frecuencia'])
  lim_monto = IQR(rfm_df['Monto'])

  cat_rec = categorizacion(rfm_df['Recencia'], lim_rec, reciencia= True)
  cat_freq = categorizacion(rfm_df['Frecuencia'], lim_freq, reciencia=False)
  cat_monto = categorizacion(rfm_df['Monto'], lim_monto, reciencia=False)

  # Campos de puntaje
  rfm_df['puntaje_recencia'] = cat_rec[0]
  rfm_df['puntaje_frecuencia'] = cat_freq[0]
  rfm_df['puntaje_monto'] = cat_monto[0]

  # Campos de rango 
  rfm_df['rango_recencia'] = cat_rec[1]
  rfm_df['rango_frecuencia'] = cat_freq[1]
  rfm_df['rango_monto'] = cat_monto[1]
  
  rfm_df['score'] = rfm_df['puntaje_recencia'].astype(str)\
     + rfm_df['puntaje_frecuencia'].astype(str)\
     + rfm_df['puntaje_monto'].astype(str)
  
  return(rfm_df)


In [2]:
# Carga de la base
inpath = "C:/Users/Diego Torres/OneDrive - Manar Technologies SAS/Carvajal/Bases/"

tb_ini = pd.read_excel(inpath + 'tb_Carvajal1.xlsx',
                       sheet_name="Hoja2")

In [3]:
#################
# Descriptivos
print(tb_ini.sample(5))
print(' ')
print('Summary:')
tb_ini.info()
print(' ')
print('Descriptivos:')
print(tb_ini.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(tb_ini.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(tb_ini.shape)

       codigo cliente  nombre cliente  codigo producto   
10032     13330695284     13330695284         32408002  \
42898     13330000912     13330000912         32105003   
47622       113202668       113202668         31000686   
58086       113299696       113299696         31000868   
66616       112388082       112388082         31705007   

                                         nombre producto fecha venta   
10032              DIF MT-IBC MENSUALIDAD CEN FINANCIERO  2022-03-23  \
42898        MT-IBC MENSUALIDAD ADMINISTRACION CENT PLUS  2022-06-02   
47622  **EXC EB - MENSUALIDAD VARIABLE FACTURA ELECTR...  2022-06-14   
58086                            **EXC EB - PYME PLAN 10  2022-07-05   
66616  DIF MANTENIMIENTO A DISTANCIA DE PROGRAMAS Y E...  2022-07-25   

       cantidad facturado  valor facturado                 Tipo  
10032                   1          45411.0       CEN FINANCIERO  
42898                   1         402403.0    CEN TRANSACCIONAL  
47622               